## An Analysis of the 2024 Syracuse Mets
In this analysis we're going to be looking at basic stats collected from Baseball Refernce and new advanced stats scraped from Baseball Savant's minor league stat cast database.
I'm going to focus on hitters with over 100 plate appearences so we can get an accurate understanding of players who have a decent sample size of play in AAA. I'll also be looking at players under the age of 27 since the Mets as an organization have a large focus on their young prospects that can help the team win in the near future.

In [1]:
import pandas as pd
from utils import *
from IPython.display import display, HTML

# You can also prevent truncation of large DataFrames
pd.set_option('display.expand_frame_repr', False)

# Set maximum column width
pd.set_option('display.max_colwidth', 200)

# Set the maximum number of columns displayed
pd.set_option('display.max_columns', None)

# Set the maximum number of rows displayed
pd.set_option('display.max_rows', None)

# Set display width to ensure DataFrame fits in the notebook
pd.set_option('display.width', 3000)

# Select Mets players below 27 and with over 100 plate appearences
completeMetsAAAStats = completeMetsAAAStats[(completeMetsAAAStats['PA'] > 100) & (completeMetsAAAStats['Age'] <= 26)]

## Initial Analysis / Basic Slash Line

To start our analysis, I'm going to look at the most traditional way we look at hitters: the slash line. Now I know OPS is not included in a traditional slash line but I put it because I believe it's one of the best non-advanced stats to display a hitter's overall performance.

With our criteria stated above, we're given 5 players on the team that meet these requirements: Brett Baty, Mark Vientos, Luisangel Acuna, Thomas Rhyland, and Drew Gilbert.

Looking at these stats I can see two standouts, Brett Baty and Mark Vientos. I say these are our standouts because they're hitting above .800 OPS demonstrating an ability to hit for power and get on base, at least on a surface level. 

Let's take a look at how these players stack up versus AAA league averages.

In [2]:
# Select the basic statline stats and basic info I want to see for each player: Player, Age, G, PA, AB, BA, OBP, SLG, OPS
print(completeMetsAAAStats[defaultInfo + slashLine].sort_values(by='OPS', ascending=False))

                             Player   Age    G   PA   AB     BA    OBP    SLG    OPS
3                       Baty, Brett  24.0   56  246  211  0.261  0.358  0.536  0.885
27                    Vientos, Mark  24.0   31  132  116  0.284  0.371  0.500  0.876
0        Acuï¿½a, Luisangel NYM #12  22.0  110  486  452  0.265  0.313  0.369  0.676
25                   Thomas, Rhylan  24.0   49  192  179  0.229  0.281  0.374  0.663
9   Gilbert, Drew MLB #81 / #NYM #3  23.0   26  111   97  0.216  0.315  0.309  0.577


## League Averages Comparison
<p>
In terms of batting average, Brett Baty sits at just above the league average. But if we continue to look down his stat line, he's anything but average. Baty's in the top half of the league for OBP which is incredibly valuable in today's game. You need base runners to score runs. However, this is not where his strangth is. His strength lies in his SLG/power. His slugging percentage is in the top quartile for the league, pretty handidly too. He's also in the top quartile for OPS. On a basic level, this shows his ability to hit for power while not compromising his ability to get on base. 

Next, I'll be looking at Mark Vientos. Although Mark Vientos has already been promoted to the Majors, it's important to look at the stats of players that have been called up and had success in the MLB. And I would definitely put Mark Vientos in this category with his incredible 2024 season. When looking at Vientos' batting average, it's clear he was having some level of success at the plate. He ranked in the upper quartile comfortably. Suprsingly, this is true for all of his stats except OBP where he still ranked in the 3rd quartile. Similar to Baty, Mark Vientos' slash line conveys his ability to hit for power but where he outperforms Baty is getting on base and getting on base through contact. However, Baty still does slightly have the edge in terms of power if we're basing this purely on slugging percentage. 

Now lets take a look at one of our top prospects: Luisangel Acuña. At first glance, his slash line is extremly underwhelming. For every stat except batting average, he ranks in the bottom quartile of the league. And even his batting average only falls into the third quartile. This tells me that despite the fact he can get on base fairly well through his hits, his overall ability to get on base as well as his power is lacking.

A lot of these observations for Acuña can made for Drew Gilbert as well thus far. They also fall into the bottom quartile for all slash line stats as well as OBP. However, with Drew Gilbert it's important to note his experience in AAA so far. He's barely had 100 plate appearences and more likely than not just needs more time to adjust to AAA. He previously put up decent numbers in the lower levels. We traded Rylan Thomas for Ryne Stanek so I won't focus on him too much.
</p>

In [3]:
# Clean and convert columns that are mistyped
columnsToConvert = ['OBP', 'SLG', 'BA']
leaguewideAAAStatsDf = convertColumnsToNumeric(leaguewideAAAStatsDf, columnsToConvert).copy()

# Calculate OPS for leaguewide data, this isn't a stat provided to us by Baseball Savant in the league data
leaguewideAAAStatsDf['OPS'] = leaguewideAAAStatsDf['OBP'] + leaguewideAAAStatsDf['SLG']

# Select all players in AAA with over 100 plate appearences
leaguewideAAAStatsDf = leaguewideAAAStatsDf[leaguewideAAAStatsDf['PA'] > 100]

# Get league mean, standard deviation, percentiles, and min/max values for BA, OBP, SLG, and OPS and concat the dataframes to one list for easy viewing. 
summary_df = pd.concat([
    leaguewideAAAStatsDf['BA'].describe(),
    leaguewideAAAStatsDf['OBP'].describe(),
    leaguewideAAAStatsDf['SLG'].describe(),
    leaguewideAAAStatsDf['OPS'].describe()
], axis=1)

print(summary_df)

               BA         OBP         SLG         OPS
count  471.000000  471.000000  471.000000  471.000000
mean     0.257062    0.347420    0.428486    0.775907
std      0.038070    0.042097    0.081750    0.112078
min      0.135000    0.204000    0.226000    0.431000
25%      0.234000    0.322000    0.374000    0.703000
50%      0.258000    0.346000    0.431000    0.780000
75%      0.280000    0.373000    0.480500    0.853500
max      0.401000    0.515000    0.678000    1.179000


## Actual Stats vs Expected Stats
Before looking at potential reasons why these player's stats look like this, it's important to see how they're peforming vs their expected stats. 

Let's examine Brett Baty. 
His actual stats are lower than his expected stats in every category. This could be the case for a number of reasons and doesn't necessarily mean he's underperforming. Since Baty's batting average is lower than his expected output, this could suggest a number of possibilities. It is very possible he's just outright underperorming, but more likely than not it means he's getting very unlucky with some hard hit balls or hitting against some difficult defensive alignments. I say this because a .278 xBA is still great and if Baty continues to hit like this, we should see his actual batting average go up. I would give a similar analysis to his OBP vs xOBP, SLG vs xSLG, and wOBA vs xwOBA. If Baty continues the quality of his at bats and contact, his slash line should continue to trend up.

Next lets look at an interesting case: Mark Vientos. Despite the fact that he's doing better than Baty in BA and OBP and slighly worse in SLG, all of his expected stats are lower than Baty's. Since Vientos' actual stats are significantly higher than his expected, this could be indicative of a few things. It could mean he's getting lucky for the quality of his contact. Maybe he's getting a few lucky bounces or bloop hits that would raise his average. Or he could be potentially hitting home runs/extra-base hits that might not always carry out of the park or fall in for hits. While this might indicate his actual stats aren't sustainable, I peronsally believe these should be taken with a grain of salt. It's important to also look at the quality of contact and power a hitter is making to come to a complete conclusion, which we'll do later on. 

Let's move on to Luisangel Acuna. Similar to Vientos, he's outpeforming his expected stats except all of his actual and expected stats are lower than Vientos'. This shows me that if Vientos is getting lucky, then Acuna is getting nearly as lucky but seeing less of a payoff for it. His average is lower, he's getting on base less, and power is lacking. So the observations we made for Vientos' contact quality and base hit luck are same that I'd make here and really for all the prospects beneath Acuna as well. 

While I think these are important stats to note and observe, I don't know if they necessarily directly translate to any concrete predictions. Even though Baty's expected stats are better than Vientos', Baty has struggled at the major league level while Vientos has seen success this season. This is not to say ignore these stats, but they are definitely more descriptive than predictive.

In [4]:
print(completeMetsAAAStats[['Player'] + actualVsEx + ['OPS']].sort_values(by='OPS', ascending=False))

                             Player     BA    xBA    OBP   xOBP    SLG   xSLG   wOBA  xwOBA    OPS
3                       Baty, Brett  0.261  0.278  0.358  0.375  0.536  0.552  0.381  0.397  0.885
27                    Vientos, Mark  0.284  0.237  0.371  0.331  0.500  0.444  0.376  0.337  0.876
0        Acuï¿½a, Luisangel NYM #12  0.265  0.236  0.313  0.287  0.369  0.325  0.299  0.273  0.676
25                   Thomas, Rhylan  0.229  0.203  0.281  0.257  0.374  0.281  0.286  0.242  0.663
9   Gilbert, Drew MLB #81 / #NYM #3  0.216  0.189  0.315  0.291  0.309  0.282  0.286  0.264  0.577


# Batter Profiles

In [5]:
# Determine columns that are getting cleaned and converted
columnsToConvert = ['Barrel/PA%', 'Barrel/BBE%', 'BABIP', 'Hard Hit%', 'EV (MPH)', 'Adj. EV (MPH)', 'Dist (ft)']
leaguewideAverages = describeColumns(leaguewideAAAStatsDf, columnsToConvert)

print(leaguewideAverages)
print(completeMetsAAAStats[['Player'] + contactQuality].sort_values(by='Barrel/BBE%', ascending=False))

       Barrel/PA%  Barrel/BBE%       BABIP   Hard Hit%    EV (MPH)  Adj. EV (MPH)   Dist (ft)
count  471.000000   471.000000  471.000000  471.000000  471.000000     471.000000  471.000000
mean     3.971550     6.336518    0.312605   35.555626   87.719108      91.218896  154.777070
std      2.194024     3.711165    0.044797    8.125097    2.313921       0.818278   11.143512
min      0.000000     0.000000    0.153000    9.400000   80.400000      89.200000  126.000000
25%      2.300000     3.400000    0.286000   30.600000   86.100000      90.600000  147.000000
50%      3.700000     5.700000    0.314000   35.800000   87.700000      91.200000  155.000000
75%      5.400000     8.800000    0.340000   41.450000   89.400000      91.700000  162.000000
max     10.700000    18.000000    0.486000   59.100000   95.500000      94.100000  184.000000
                             Player  BIP  BABIP  Barrels  Hard Hit%  EV (MPH)  Adj. EV (MPH)  LA (ï¿½)  Dist (ft)  Barrel/BBE%  Barrel/PA%
3              

## Brett Baty
Let's take a look back at the profile we've developed for Brett Baty before proceeding. Brett Baty is someone who can get on base at an above average rate but his strength lies in his slugging. He's excellent at hitting for power. However, after looking at his expected stats, it seems like he's getting unlucky on some hard hit balls or defensive alignments, leading to lower than expected hitting stats. But if he continues to hit like he is right now, his slash line should go up.

To confirm this I'll look at his BABIP, Hard Hit%, adjusted exit velo, as well as both of his barrel ratios. In terms of BABIP, Baty falls into the bottom quartile meaning that whenever he puts the ball in play, he's getting on base at one of the lowest rates in the league. Now a low BABIP can be an indicator for a number of things. Bad luck, weak contact, lots of shift adjusments, or even slow on the bases. Looking at some other stats available, we can take out the possibility of weak contact. Baty's Hard Hit%, adjusted exit velo, hit distance, and barrel ratios are all in the top quartile. Meaning he hits the ball hard at one of the highest rates in the league. For non adjusted exit velo he also still falls in the top half of the league, just missing the cut off for the upper quartile. This almost perfectly fits the description we came up with for Baty. He hits the ball extremely hard, but on balls in play he can get unlucky, hit into the shift, or he's slow on the base path. 

Luckily, with Baseball Savant's minor league statcast search, we can look at pitch-by-pitch statistics.

In [6]:
# Modify the column name for pitch type
metsAAApbpSavant.columns.values[0] = 'pitch_type'

# Clean and convert 
colToCon = ['launch_speed']
batyPbp = convertColumnsToNumeric(metsAAApbpSavant, colToCon)

# Select info for Brett Baty
batyPbp = batyPbp[(batyPbp['player_name'] == 'Baty, Brett')]

# Select pitches were the outcome was an out in play
batyPbpOuts = batyPbp[batyPbp['events'].isin(pbpOuts)].copy()

# Select outs that were a barrel or hard hit outs Baty Had
hardHitOuts = batyPbpOuts[(batyPbpOuts['player_name'] == 'Baty, Brett') & (batyPbpOuts['launch_speed'] >= 95)].copy()
barrelOuts = batyPbpOuts[hardContact][(batyPbpOuts['launch_speed'] >= 95) & (batyPbpOuts['launch_angle'] >= 26)].copy()

#print(batyPbp[hardContact].sort_values(by='launch_speed', ascending=False))
print(barrelOuts)
print('Hard hard outs: ' + str(len(hardHitOuts)))
print('Barrel outs: ' + str(len(barrelOuts)))

      player_name   game_date pitch_type  launch_angle  launch_speed  hit_location  hit_distance_sc     events                                                             des
105   Baty, Brett  2024-08-18         CU          36.0          96.5           7.0            337.0  field_out                  Brett Baty flies out to left fielder Jake Alu.
2716  Baty, Brett  2024-07-27         FF          26.0         100.5           9.0            354.0  field_out   Brett Baty flies out sharply to right fielder Will Robertson.
5933  Baty, Brett  2024-06-26         FC          34.0         101.2           8.0            396.0  field_out  Brett Baty flies out sharply to center fielder Will Robertson.
6753  Baty, Brett  2024-06-19         FC          63.0         100.6           7.0            193.0  field_out                 Brett Baty flies out to left fielder Alex Call.
Hard hard outs: 28
Barrel outs: 4


Unfortunately, I can't get descriptive league wide stats on how many hard hit outs each hitter has due to some Baseball Savant limitations. If I try to get league wide pitch by pitch stats, the pitches only go back to mid-July and I'd have to scrape the rest of the dates/pitches dynamically which posed its own set of problems. Ultimately, it seemed like more work then necessary for something I would use once. However, in the future as I expand this project I will be looking to solve this issue.

I was able to do a similar search using Baseball Savant's filters and found out Baty was in the bottom half of hitters with barrel/hard hit outs. So it doesn't seem like he's getting anymore unlucky than his peers.

This isn't particularly important to Baty's analysis, but if we look at his hard hit out from June 26th, 2024... How did this not leave the park? 101.2 launch speed and it traveled 396 feet. 

In [7]:
print(len(batyPbpOuts))
print(batyPbpOuts['hit_location'].value_counts())

105
4.0    26
3.0    16
6.0    16
7.0    15
5.0    10
8.0    10
9.0     5
1.0     4
2.0     3
Name: hit_location, dtype: int64


Looking at the location of his outs, we find why Baty's BABIP and BA is lower than expected: he's hitting defensive shifts. So far this season he's had 105 outs in play and 42 of these were to the first basemen or second basemen, which is about 40%. If we expand this to include the shortstop who is very often involved in shifts, it would go up to 58 outs into the shift, or 55% of Baty's in-play outs.